# Setting up NDN-DPDK on FABRIC with a complex topology

## Slice Provisioning
First, we set up fablib so it can be used in the rest of the notebook.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

Next, we select a site with enough resources for the topology.

In [ ]:
cores_per_node = 16
ram_per_node = 32
disk_per_node = 20 # we will modify this later to give fileservers more
site_name = fablib.get_random_site(
    filter_function=lambda x: x["cores_available"] > cores_per_node * 10 and x["ram_available"] > ram_per_node * 10 and x["disk_available"] > disk_per_node * 10
)

We then find our slice or create one if we don't have it yet.

In [ ]:
import os
slice_name="IP_NDN_Test_" + os.getenv('NB_USER')

try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
except:
    print("You don't have a slice named %s yet." % slice_name)
    slice = fablib.new_slice(name=slice_name)

Next we name and set the networks and interfaces<br>

In [ ]:
# this cell sets up the hosts and router
node_names = ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]
for n in node_names:
    node = slice.add_node(name=n, site=site_name, cores=16, ram=32, disk=20, image='default_ubuntu_22')
    node.add_post_boot_upload_directory("scripts")
    #node.add_post_boot_execute("./scripts/install_dependencies.sh")
    node.add_post_boot_execute("./scripts/install-ndn-dpdk.sh")
    
# this cell sets up the network links
nets = [
    {"name": "net1",   "nodes": ["r1","c1","c2"]},
    {"name": "net2",   "nodes": ["r2","p1"]},
    {"name": "net3",   "nodes": ["r3","p2"]},
    {"name": "net4",   "nodes": ["r4","p3","c3"]},
    {"name": "net12",   "nodes": ["r1","r2"]},
    {"name": "net23",   "nodes": ["r2","r3"]},
    {"name": "net34",   "nodes": ["r3","r4"]},
    {"name": "net41",   "nodes": ["r4","r1"]},
]
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

<br><br>Request the slice and wait until it has been reserved<br>

In [ ]:
slice.submit()

## NDN Setup:

First, we declare a bunch of variables to use later:

In [ ]:
hosts = ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]

# this map is used to set up the arguments for the forwarder roles
FW_ARGS = {
  'mempool': {
    'DIRECT': { 'capacity': 524287, 'dataroom': 9146 },
    'INDIRECT': { 'capacity': 524287 },
  },
  'fib': {
    'capacity': 4095,
    'startDepth': 8,
  },
  'pcct': {
    'pcctCapacity': 65535,
    'csMemoryCapacity': 20000,
    'csIndirectCapacity': 20000,
  }
}
# this map is used to configure the memif interface for the producer roles
MEMIF_ARGS = {
    'scheme': "memif",
    'socketName': "/run/ndn/fileserver.sock",
    'id': 0,
    'role': "server",
    'dataroom': 9000
}

# this helper function configures and sets up the namespaces for the producer roles
def make_fs_activate_json(node):
    activate_json = {
        'eal': {
            'memPerNuma': {'0': 4*1024},
            'filePrefix': 'producer',
        },
        'mempool': {
            'DIRECT': {'capacity': 2**16-1, 'dataroom': 9200},
            'INDIRECT': {'capacity': 2**16-1},
            'PAYLOAD': {'capacity': 2**16-1, 'dataroom': 9200},
        },
        'face': {
            'scheme': 'memif',
            'socketName': MEMIF_ARGS['socketName'],
            'id': MEMIF_ARGS['id'],
            'dataroom': 9000,
            'role': 'client',
        },
        'fileServer': {
            'mounts': [
                {'prefix': f"/{node}/usr-local-share", 'path': "/usr/local/share"}
            ],
            'segmentLen': 6 * 1024,
        },
    }
    return activate_json

# convenience maps for the network
net_to_nodes = {
    "net1": ["r1","c1","c2"],
    "net2": ["r2","p1"],
    "net3": ["r3","p2"],
    "net4": ["r4","p3","c3"],
    "net12": ["r1","r2"],
    "net23": ["r2","r3"],
    "net34": ["r3","r4"],
    "net41": ["r4","r1"],
}

adj_list = {
    "r1": ["c1","c2","r2","r4"],
    "r2": ["p1","r1","r3"],
    "r3": ["p2","r2","r4"],
    "r4": ["p3","c3","r1","r3"],
    "c1": ["r1"],
    "c2": ["r1"],
    "c3": ["r4"],
    "p1": ["r2"],
    "p2": ["r3"],
    "p3": ["r4"],
}
consumers = ["c1","c2","c3"]
routers = ["r1","r2","r3","r4"]
producers = ["p1", "p2", "p3"]

In [ ]:
# these maps store the output of the NDN-DPDK creation calls
forwarder_hashes = {}
face_hashes = {}
node_net_interface = {}

This cell determines the network interface names for each of the nodes. This is especially important for the router nodes.

In [ ]:
import json
interfaces = json.loads(slice.list_interfaces(output="json"))
for i in interfaces:
    h = i["node"]
    net = i["network"]
    inf = i["physical_dev"]
    try:
        node_net_interface[h]
    except KeyError:
        node_net_interface[h] = {}
    node_net_interface[h][net] = inf
print(node_net_interface)

This cell sets up the ethernet ports on the consumers and producers. This is relatively simple as each of these nodes only points to their connected router node.

In [ ]:
import shlex
import json

# add eth ports to each node
for h in (consumers + producers):
    node = slice.get_node(h)
    # consumers and producers only have 1 interface
    inf = next(iter(node_net_interface[h].values()))
    output = node.execute(f'''
        echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif {inf} --xdp --mtu 1500
        ''')
    forwarder_hashes[h] = json.loads(output[0].split("\n")[-2])

This cell sets up each router node to connect to each neighboring router node as well as the consumers/producers it is connected to.

You will receive "ndndpdk-svc is already activated" warnings while running this cell. That is expected as each router runs through this multiple times.

In [ ]:
for h in routers:
    forwarder_hashes[h] = {}
    for net, inf in iter(node_net_interface[h].items()):
        node = slice.get_node(h)
        output = node.execute(f'''
            echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
            ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif {inf} --xdp --mtu 1500
            ''')
        forwarder_hashes[h][net] = json.loads(output[0].split("\n")[-2])

This cell sets up all the ethernet faces for the NDN-DPDK instances on all the routers and also does it for each connected endpoint to each router.

In [ ]:
for h in routers:
    node = slice.get_node(h)
    for net, output in iter(forwarder_hashes[h].items()):
        local_mac = output['macAddr']
        for r in net_to_nodes[net]:
            if h == r:
                continue
            else:
                print(f"h: {h}, r: {r}, net: {net}")
                print(f"local_mac: {local_mac}")
                if r in routers:
                    remote_mac = forwarder_hashes[r][net]['macAddr']
                    print(f"remote_mac: {remote_mac}")
                else:
                    remote_mac = forwarder_hashes[r]['macAddr']
                    print(f"remote_mac: {remote_mac}")
                    # do the reverse since we have the correct mac address here
                    remote_node = slice.get_node(r)
                    out2 = remote_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {remote_mac} --remote {local_mac}")
                    face_hashes[f"{r}:{h}"] = json.loads(out2[0])
                # we only do 1 way for routers because we're going to do the other side in another loop
                out = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
                face_hashes[f"{h}:{r}"] = json.loads(out[0])

This cell sets up memif faces for each producer as well as activating a fileserver instance for each one.

In [ ]:
for p in producers:
    producer_node = slice.get_node(p)
    output = producer_node.execute(f'''
        MEMIF_FACE=$(echo {shlex.quote(json.dumps(MEMIF_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-face)
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh $(echo $MEMIF_FACE | jq -r .id)
        ''')
    # start fileserver instance
    output = producer_node.execute(f'''
        sudo ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/ systemd start
        echo {shlex.quote(json.dumps(make_fs_activate_json(p)))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3031 activate-fileserver
        ''')

This cell sets up the FIB entries on each router to point to their attached producer

In [ ]:
for p in producers:
    for r in adj_list[p]:
        node = slice.get_node(r)
        c_to_r = face_hashes[f"{r}:{p}"]
        print(c_to_r)
        output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh {c_to_r['id']}")

This cell sets up the FIB entries on each consumer to point to their attached router for each producer

In [ ]:
for c in consumers:
    for r in adj_list[c]:
        node = slice.get_node(c)
        c_to_r = face_hashes[f"{c}:{r}"]
        for p in producers:
            output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh {c_to_r['id']}")

This cell sets up FIB entries on each router to point to their neighboring routers for each producer they aren't directly attached to.

In [ ]:
for r in routers:
    node = slice.get_node(r)
    router_hops = []
    for h in adj_list[r]:
        if h in routers:
            router_hops.append(face_hashes[f"{r}:{h}"]['id'])
    for p in producers:
        if p in adj_list[r]:
            continue
        else:
            output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh {' --nh '.join(router_hops)}")

Once all the NDN-DPDK configuration is done, we create test files on each producer node.

In [ ]:
for p in producers:
    node = slice.get_node(p)
    output = producer_node.execute(f'''
        cd /usr/local/share
        sudo head -c 500K </dev/urandom >500Ktestfile
        sudo cp 500Ktestfile 500Ktestfile2
        sudo cp 500Ktestfile 500Ktestfile3
        sudo cp 500Ktestfile 500Ktestfile4
        sudo cp 500Ktestfile 500Ktestfile5
        sudo head -c 100M </dev/urandom >100Mtestfile
        sudo cp 100Mtestfile 100Mtestfile2
        sudo cp 100Mtestfile 100Mtestfile3
        sudo cp 100Mtestfile 100Mtestfile4
        sudo cp 100Mtestfile 100Mtestfile5
    ''')

Once all the files have been created, we can begin testing their transfer!

SSH into a consumer node and run the following command to transfer a file from a producer of your choosing to that consumer (you can swap out the file names for any existing file):

`sudo ndndpdk-godemo fetch --filename /tmp/myfile --name /p1/usr-local-share/500Ktestfile`

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 3 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

slice.update()
_ = slice.show()